# What is about ?

Preliminary draft

Baseline to decompose element into product of generators by ML methods. Permutation groups. 

    - Generate train set by random walks , i.e. pairs (g, k) 
    - Train model to preduct k from g
    - Use it as heuristics in any algorithm which finds global minima of distnace, e.g. Metpropolis
    
    



# Example to use one-hot  encoding to convert permutations from vector format to permutation-matrix format

https://en.wikipedia.org/wiki/Permutation_matrix

## Prepare 

## Examples 

# Preliminaries 

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
from tqdm.notebook import tqdm
import time
t0start = time.time() 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
import seaborn as sns


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import warnings
warnings.filterwarnings('ignore')
import os
# i = 0
# for dirname, _, filenames in os.walk('/kaggle/input'):
# #     if dirname == '/kaggle/input/growth-in-finite-groups/UT4_growth':
#     for filename in filenames:
#         i+=1
#         if i<= 1:
#             print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 

In [2]:
!ls /kaggle/input/growth-in-finite-groups

'Heis 3 gens data'
 HeisenbergComputations.sage
'Rationality of growths of groups Ho.pdf'
 UT4_growth
 UT4_states_and_distances
 UT5_growth
 UT5_states_and_distances
 UT6_growth
 UT6_states_and_distances
 UT7_growth
 UT8_growth
 cubes444_Santa_states_and_n_best_moves.csv
 empty.txt
 puzzle_info.csv
 puzzles_best_scores_numeric_states_etc.csv


In [3]:
import torch 
import math

if torch.cuda.is_available():
    device = torch.device("cuda")
    dtype = torch.float
else:
    device = torch.device("cpu")
    dtype = torch.float#int16  
    dtype = torch.int16  
    
print(dtype)    
print(device)


torch.float32
cuda


# Preliminaries Groups data 

## Load moves data 

In [4]:
import numpy as np
import pandas as pd 
# # fn = '/kaggle/input/santa-2023/puzzle_info.csv'
# fn = '/kaggle/input/growth-in-finite-groups/puzzle_info.csv'
# di = pd.read_csv(fn)
# di['n_generators'] = di['allowed_moves'].apply(lambda x: len(x.split(':'))-1)
# di['vector_len'] = di['allowed_moves'].apply(lambda x: len(x.split(']')[0].split('[')[1].split(',')))
# di['puzzle_type_brief'] = di['puzzle_type'].apply(lambda x: x.split('_')[0])

# display(di)
# di.to_csv('moves.csv')

In [5]:
1

1

In [6]:
def random_walks(n_random_walk_length,  n_random_walks_to_generate):
      '''
      Output:
      returns X,y: X - array of states, y - number of steps rw achieves it

      Input:
      generators - generators (moves) to make random walks  (permutations),
          can be list of vectors or array with vstacked vectors
      n_random_walk_length - number of visited nodes, i.e. number of steps + 1
      n_random_walks_to_generate - how many random walks will run in parrallel
      rw_start - initial states for random walks - by default we will use 0,1,2,3 ...
          Can be vector or array
          If it is vector it will be broadcasted n_random_walks_to_generate times,
          If it is array n_random_walks_to_generate - input n_random_walks_to_generate will be ignored
              and will be assigned: n_random_walks_to_generate = rw_start.shape[0]

      '''
      state_size        = len(list_generators[0])
      n_generators      = len(list_generators  )
      dtype_generators = torch.int64
      tensor_generators = torch.tensor(   list_generators       , device = device, dtype = dtype_generators, )
      state_destination = torch.arange( state_size, device=device, dtype = dtype).reshape(-1,state_size)
      array_of_states = state_destination.to(device).view( 1, state_size  ).expand( n_random_walks_to_generate, state_size )

      # Output: X,y - states, y - how many steps we achieve them
      # Allocate memory:
      X = torch.zeros( n_random_walks_to_generate*n_random_walk_length , state_size, device=device, dtype = dtype )

      # First portion of data  - just our state_rw_start state  multiplexed many times
      X[:n_random_walks_to_generate,:] = array_of_states
      y        = torch.tensor(range(n_random_walks_to_generate*n_random_walk_length), device=device)//n_random_walks_to_generate
      IX_moves = torch.randint(0, n_generators, (n_random_walks_to_generate*n_random_walk_length-n_random_walks_to_generate,), dtype = torch.int32, device=device)

      # Technical to make array[ IX_array] we need  actually to write array[ range(N), IX_array  ]

      # Main loop

      argnr = (torch.arange(n_random_walks_to_generate, dtype=torch.long, device=device)*torch.ones((n_random_walks_to_generate,tensor_generators.shape[1]), dtype=torch.long, device=device).T).T

      for i_step in range(1,n_random_walk_length):
          a,b,c = (i_step-1)*n_random_walks_to_generate, (i_step-0)*n_random_walks_to_generate, (i_step+1)*n_random_walks_to_generate
          #X[ b:c, : ] = X[ a:b, : ][argnr, self.tensor_generators[IX_moves[ a:b ],:]]
          X[ b:c, : ] = torch.gather( X[ a:b, : ], 1, tensor_generators[IX_moves[ a:b ],:] )

      return X,y

In [7]:
def random_walks(generators , n_random_walk_length,  n_random_walks_to_generate,    state_rw_start = 'Auto',
                 n_steps_back_to_ban = 0, walks_type = 'non-backtracking-beam',
                 device='Auto', dtype = float , vec_hasher = 'Auto', verbose = 0   ):
    '''
    Output:
    returns X,y: X - array of states, y - number of steps rw achieves it

    Input:
    generators - generators (moves) to make random walks  (permutations),
        can be list of vectors or array with vstacked vectors
    n_random_walk_length - number of visited nodes, i.e. number of steps + 1
    n_random_walks_to_generate - how many random walks will run in parrallel
    state_rw_start - initial states for random walks - by default we will use 0,1,2,3 ...
        Can be vector or array
        If it is vector it will be broadcasted n_random_walks_to_generate times,
        If it is array n_random_walks_to_generate - input n_random_walks_to_generate will be ignored
            and will be assigned: n_random_walks_to_generate = rw_start.shape[0]

    '''
    t0 = time.time()
    ##########################################################################################
    # Processing/Reformating input params
    ##########################################################################################

    # device
    if device == 'Auto':
        if torch.cuda.is_available():
            device = torch.device("cuda")
        else:
            device = torch.device("cpu")
    # Analyse input format of "generators"
    # can be list_generators, or tensor/np.array with rows - generators
    if isinstance(generators, list):
        list_generators = generators
    elif isinstance(generators, tuple):
        list_generators = list(generators)
    elif isinstance(generators, torch.Tensor ):
        list_generators = [ list(generators[i,:]) for i in range(generators.shape[0] ) ]
    elif isinstance(generators, np.ndarray ):
        list_generators = [list(generators[i,:]) for i in range(generators.shape[0] ) ]
    else:
        print('Unsupported format for "generators"', type(generators), generators)
        raise ValueError('Unsupported format for "generators" ' + str(type(generators)) )
    state_size = len(list_generators[0])
    n_generators = len( list_generators )

    # dtype
    if (state_rw_start == '01234...') or (state_rw_start == 'Auto' ):
        n_unique_symbols_in_states = state_size
    else:
        tmp = set( [int(i) for i in state_rw_start ]  ) # Number of unique elements in any iterator
        n_unique_symbols_in_states = len(tmp)
    if dtype == 'Auto':
        if n_unique_symbols_in_states <= 256:
            dtype = torch.uint8
        else:
            dtype = torch.uint16

    # Destination state
    if (state_rw_start == '01234...') or (state_rw_start == 'Auto' ):
        state_rw_start = torch.arange( state_size, device=device, dtype = dtype).reshape(-1,state_size)
    elif isinstance(state_destination, torch.Tensor ):
        state_rw_start =  state_destination.to(device).to(dtype).reshape(-1,state_size)
    else:
        state_rw_start = torch.tensor( state_destination, device=device, dtype = dtype).reshape(-1,state_size)

    # Reformat generators in torch 2d array
    dtype_generators = torch.int64
    tensor_generators = torch.tensor( list_generators , device = device, dtype =  dtype_generators  )
    #print('tensor_generators.shape:', tensor_generators.shape)

    # Preprare vector which will used for hashing - to avoid revisiting same states
    max_int =  int( (2**62) )
    dtype_for_hash = torch.int64
    if vec_hasher == 'Auto':
        vec_hasher = torch.randint(-max_int, max_int+1, size=(state_size,), device=device, dtype=dtype_for_hash)


    ##########################################################################################
    # Initializations
    ##########################################################################################

    # Main variable in the loop - store current states: 2d torch tensor
    # Initialization via state_rw_start state - duplicatie it many (n_random_walks_to_generate) times
    array_current_states = state_rw_start.view(1, state_size  ).expand(n_random_walks_to_generate, state_size ).clone()

    # Output: X,y - states, y - how many steps we achieve them
    # Allocate memory:
    X = torch.zeros( (n_random_walks_to_generate)*n_random_walk_length , state_size, device=device, dtype = dtype )
    y = torch.zeros( (n_random_walks_to_generate)*n_random_walk_length , device=device, dtype = dtype )
    # First portion of data  - just our state_rw_start state  multiplexed many ( n_random_walks_to_generate ) times
    X[:n_random_walks_to_generate,:] = array_current_states
    y[:n_random_walks_to_generate] = 0

    # Hash initial states.
    if n_steps_back_to_ban > 0:
        hash_initial_state = torch.sum( state_rw_start.view(-1, state_size  ) * vec_hasher, dim=1) # Compute hashes
        vec_hashes_current = hash_initial_state.expand( n_random_walks_to_generate * n_generators  , n_steps_back_to_ban ).clone()
        # That is equivalent to matmul, but matmul is not supported for ints in torch (on GPU <= 2018) - that is way round
        # Intialize index for hash storage
        # Newly obtained hash vectors will be stored in 2d array vec_hashes_current
        # The position/column for storage: i_cyclic_index_for_hash_storage
        i_cyclic_index_for_hash_storage = 0 # Will be updated modula n_steps_back_to_ban, i.e. from 0 to n_steps_back_to_ban-1


    if verbose >= 100:
        print('X.shape:',X.shape, 'y.shape:',y.shape)
        print(array_current_states.shape)
        print( array_current_states[:3,:] )

    ##########################################################################################
    # Main loop
    # 1. Create new states from current making ALL possible moves - thus number of states will be more than we need
    # 2. Select those states which were not visited on "NNN" previous steps. To make it fast we uses hashes:
    # 2.1. Compute hashes of these states just by scalar multiplication on random hash vector - get single int64 as a hash
    # 2.2. Choose only those states which hashes are new - not in the stored history of hashes
    # 3. Select only desired number of states - random subset of desired size: n_random_walks_to_generate subset
    # 4. Store these states into output X,y
    # 5. Update hash storage
    ##########################################################################################
    for i_step in range(1,n_random_walk_length):
        t_moves = t_hash = t_isin =  0; t_full_step = time.time() # Time profiling
        t_unique_els = 0 # not used currently

        # 1 Create new states:
        # Apply all generators to all current states at once
        # array_new_states: 2d array (n_random_walks_to_generate * n_generators  ) x state_size
        t1 = time.time()
        array_new_states = get_neighbors(array_current_states,tensor_generators  ).flatten(end_dim=1) # Flatten converts 3d array to 2d
        t_moves += (time.time() - t1)

        # 2.1 Compute hashes
        # Compute hash. For non-backtracking - selection not visited states before.
        t1 = time.time()
        vec_hashes_new = torch.sum(array_new_states * vec_hasher, dim=1) # Compute hashes
        # That is equivalent to matmul, but matmul is not supported for ints in torch (on GPU <= 2018) - that is way round
        t_hash += (time.time() - t1)
        #  print('hashed', t_hash )

        if n_steps_back_to_ban > 0:
            # 2.2 Select only states not seen before
            # Nonbacktracking - select states not visited before
            t1 = time.time()
            mask_new = ~torch.isin(vec_hashes_new, vec_hashes_current.view(-1), assume_unique=False)
            t_isin += (time.time() - t1)
            mask_new_sum = mask_new.sum().item()
            if mask_new_sum >= n_random_walks_to_generate:
                # Select only new states - not visited before
                array_new_states = array_new_states[mask_new,:]
            else:
                # Exceptional case - should not happen for large group of interest
                # The case: can not find enough new states - will take old also not to crash the code
                i_loc = n_random_walks_to_generate - mask_new_sum
                array_new_states = torch.cat((array_new_states[mask_new,:], array_new_states[~mask_new,:][:i_loc,:]), dim=0)

        # 3. Select only desired number of states
        # Select only n_random_walks_to_generate (with preliminary shuffling)
        # Update current states with them
        perm = torch.randperm(array_new_states.size(0), device = device)
        array_current_states = array_new_states[perm][:n_random_walks_to_generate]

        # 4. Store results in final output
        # print(i_step, n_random_walks_to_generate)
        y[ (i_step)*n_random_walks_to_generate : (i_step+1)*n_random_walks_to_generate  ] = i_step
        X[ (i_step)*n_random_walks_to_generate : (i_step+1)*n_random_walks_to_generate , : ] = array_current_states.float()

        if n_steps_back_to_ban>0:
            # 5. Update hash storage
            # Pay attention - we store hashes for ALL obtained states not only for those selected - that gives us improvement:
            # We improve the chances that states obtaine on i_step will be on true graph distance i_step - our ideal goal.
            # Which might not always be the case since random walk may create loops.
            # All the states which are achieved - they need not more than i_step steps - so it is better to ban them all
            # Thus we improve chances that the next states will increase the true graph distance
            i_cyclic_index_for_hash_storage = (i_cyclic_index_for_hash_storage + 1 ) % n_steps_back_to_ban
            vec_hashes_current[:, i_cyclic_index_for_hash_storage ] = vec_hashes_new

        if verbose >= 10:
            t_full_step = time.time()-t_full_step
            print(i_step,'i_step', 'array_current_states.shape:',array_current_states.shape, 'Time %.3f'%(time.time()-t0),
                 't_moves  %.3f, t_hash  %.3f, t_isin %.3f, t_unique_els  %.3f, t_full_step %.3f'%(t_moves ,
                  t_hash , t_isin , t_unique_els, t_full_step) )

    return X,y
def get_neighbors(states, moves):
    """
    Some torch magic to apply all moves to all states at once
    Input:
    states: 2d torch array n_states x n_state_size - rows are states-vectors
    moves (int64): 2d torch array n_moves x  n_state_size - rows are permutations describing moves
    Returns:
    3d tensor all moves applied to all states, shape: n_states x n_moves x n_state_size
    Typically output is followed by .flatten(end_dim=1), which flattens to 2d array ( n_states * n_moves) x n_state_size
    """
    return torch.gather(
        states.unsqueeze(1).expand(states.size(0), moves.shape[0], states.size(1)),
        2,
        moves.unsqueeze(0).expand(states.size(0), moves.shape[0], states.size(1)))

In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score
import torch
import torch.nn as nn
import torch.nn.functional as F
import copy
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import numpy as np

In [9]:


class MatrixDatasetRegrSmall(Dataset):
    def __init__(self, X):
        self.X = torch.tensor(X, dtype=torch.int64)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        x = F.one_hot(self.X[idx], num_classes=self.X.shape[1]).float()
        return x.unsqueeze(0)

class MyIntSequenceDatasetSmall(Dataset):
    def __init__(self, X, is_regression=True):
        # Store integer inputs and targets
        dtype_y = torch.float32 if is_regression else torch.long
        self.X = torch.tensor(X, dtype=torch.long)  # shape (N, 96)
        
        self.is_regression = is_regression

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        # Return (input_sequence, target)
        return self.X[idx]
class MatrixDatasetRegr(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.int64)
        self.y = torch.tensor(y, dtype=torch.float32)

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        x = F.one_hot(self.X[idx], num_classes=self.X.shape[1]).float()
        return x.unsqueeze(0), self.y[idx]

class MyIntSequenceDataset(Dataset):
    def __init__(self, X, y, is_regression=True):
        # Store integer inputs and targets
        dtype_y = torch.float32 if is_regression else torch.long
        self.X = torch.tensor(X, dtype=torch.long)  # shape (N, 96)
        self.y = torch.tensor(y, dtype=dtype_y)     # shape (N,)
        
        self.is_regression = is_regression

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        # Return (input_sequence, target)
        return self.X[idx], self.y[idx]

def make_predictions_on_array(model, array):
    model.eval()
    all_preds = []
    dataset = MyIntSequenceDatasetSmall(array)
    loader = DataLoader(dataset, batch_size=64, shuffle=False)
    with torch.no_grad():
        for batch_X in loader:
            batch_X = batch_X.to(device)
            outputs = model(batch_X)
            all_preds.extend(outputs.cpu().numpy())
    
    all_preds = np.array(all_preds)
    
    return all_preds

def calculate_metrics(model, data_loader):
    model.eval()
    all_preds = []
    all_targets = []
    with torch.no_grad():
        for batch_X, batch_y in data_loader:
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)
            outputs = model(batch_X)
            all_preds.extend(outputs.cpu().numpy())
            all_targets.extend(batch_y.cpu().numpy())
    
    all_preds = np.array(all_preds)
    all_targets = np.array(all_targets)
    
    rmse = np.sqrt(mean_squared_error(all_targets, all_preds))
    r2 = r2_score(all_targets, all_preds)
    
    return rmse, r2

In [10]:
class MatrixDatasetRegr(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.int64)
        self.y = torch.tensor(y, dtype=torch.float32)

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        x = F.one_hot(self.X[idx], num_classes=self.X.shape[1]).float()
        return x.unsqueeze(0), self.y[idx]

class MyIntSequenceDataset(Dataset):
    def __init__(self, X, y, is_regression=True):
        # Store integer inputs and targets
        dtype_y = torch.float32 if is_regression else torch.long
        self.X = X.to(torch.long)
        self.y = y.to(dtype_y)
#         self.X = torch.tensor(X, dtype=torch.long)  # shape (N, 96)
#         self.y = torch.tensor(y, dtype=dtype_y)     # shape (N,)

        self.is_regression = is_regression

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        # Return (input_sequence, target)
        return self.X[idx], self.y[idx]

# train_dataset = MyIntSequenceDataset(X_train, y_train)
# test_dataset = MyIntSequenceDataset(X_test, y_test)
# train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
# test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)
# model = CNNRegr().to(device)
# criterion = nn.MSELoss()
# optimizer = optim.Adam(model.parameters(), lr=0.001)

def calculate_metrics(model, data_loader):
    model.eval()
    all_preds = []
    all_targets = []
    with torch.no_grad():
        for batch_X, batch_y in data_loader:
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)
            outputs = model(batch_X)
            all_preds.extend(outputs.cpu().numpy())
            all_targets.extend(batch_y.cpu().numpy())

    all_preds = np.array(all_preds)
    all_targets = np.array(all_targets)

    rmse = np.sqrt(mean_squared_error(all_targets, all_preds))
    r2 = r2_score(all_targets, all_preds)

    return rmse, r2

In [11]:
import gc
from datetime import datetime
def train(num_epochs=2, model_prefix='', batch_size=128, patience=5):
  # Training loop
  train_rmse_list, train_r2_list = [], []
  test_rmse_list, test_r2_list = [], []
  loss_list = []


  best_test_rmse = float('inf')
  patience = patience
  no_improve = 0
  best_model_weights = copy.deepcopy(model.state_dict())

  for epoch in range(num_epochs):
      X_train, y_train = random_walks(list_generators, n_random_walk_length,  n_random_walks_to_generate,
                                     n_steps_back_to_ban=8)

      train_dataset = MyIntSequenceDataset(X_train, y_train)
      train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
      model.train()
      for batch_X, batch_y in train_loader:
          batch_X, batch_y = batch_X.to(device), batch_y.to(device)

          optimizer.zero_grad()
          outputs = model(batch_X)
          loss = criterion(outputs, batch_y)
          loss.backward()
          optimizer.step()

      # Calculate metrics for train and test data
      train_rmse, train_r2 = calculate_metrics(model, train_loader)
      test_rmse, test_r2 = calculate_metrics(model, test_loader)
      train_rmse_list.append(train_rmse)
      train_r2_list.append(train_r2)
      test_rmse_list.append(test_rmse)
      test_r2_list.append(test_r2)

      scheduler.step(test_rmse)

      print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Time: {str(datetime.now())}')
      print(f'Train - RMSE: {train_rmse:.4f}, R²: {train_r2:.4f}')
      print(f'Test  - RMSE: {test_rmse:.4f}, R²: {test_r2:.4f}')
      del train_dataset, train_loader
      gc.collect()
      # Early stopping
      if test_rmse < best_test_rmse:
          best_test_rmse = test_rmse
          no_improve = 0
          torch.save(model.state_dict(), f'best_model_{model_prefix}_{best_test_rmse}.pth')
          best_model_weights = copy.deepcopy(model.state_dict())
      else:
          no_improve += 1
          if no_improve == patience:
              print("Early stopping triggered")
              break
  print('Finished Training')
  print(f'Best metrics: {min(test_rmse_list)=:.4f} {max(test_r2_list)=:.4f}')
  model.load_state_dict(best_model_weights)
  return model, train_rmse_list, train_r2_list, test_rmse_list, test_r2_list, loss_list

In [12]:
def make_S_N(N):
    return [
        [1,0,]+[q+2 for q in range(N-2)]       ,
               [q+1 for q in range(N-1)] + [0,],
        [N-1,]+[q+0 for q in range(N-1)]       ,
    ]


def make_predictions_on_array(model, array):
    model.eval()
    all_preds = []
    dataset = MyIntSequenceDatasetSmall(array)
    loader = DataLoader(dataset, batch_size=64, shuffle=False)
    with torch.no_grad():
        for batch_X in loader:
            batch_X = batch_X.to(device)
            outputs = model(batch_X)
            all_preds.extend(outputs.cpu().numpy())

    all_preds = np.array(all_preds)

    return all_preds

In [13]:
from collections import deque

def beam_search_sorting(permutation, beam_width=5):
    n = len(permutation)
    target = list(range(0, n))
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    def average_neighbor_difference(perm):
        total_diff = 0

        # Считаем разницы между соседними элементами
        for i in range(n-1):
            total_diff += abs(perm[i] - perm[i+1])

        # Добавляем разницу между последним и первым элементом
        total_diff += abs(perm[-1] - perm[0])

        return total_diff / n

    def count_monotonic_sections(perm):
        # Создаем расширенную перестановку для учета цикличности
        extended_perm = perm + perm[0:1]

        sections = 0
        increasing = None

        for i in range(n):
            if increasing is None:  # Первое сравнение
                increasing = extended_perm[i] < extended_perm[i + 1]
            elif increasing and extended_perm[i] > extended_perm[i + 1]:  # Смена возрастания на убывание
                sections += 1
                increasing = False
            elif not increasing and extended_perm[i] < extended_perm[i + 1]:  # Смена убывания на возрастание
                sections += 1
                increasing = True

        return sections / n

    def apply_actions(state):
        # Generate new states based on the allowed actions
        states = []
        # 1. Cyclic shift right
        new_state = state[-1:] + state[:-1]
        states.append((new_state, "R"))

        # 2. Cyclic shift left
        new_state = state[1:] + state[:1]
        states.append((new_state, "L"))

        # 3. Swap first two elements
        if len(state) > 1:
            new_state = state[:]
            new_state[0], new_state[1] = new_state[1], new_state[0]
            states.append((new_state, "X"))

        return states

    def apply_action(state, action):
        # 1. Cyclic shift right
        if action == 'R':
            new_state = state[-1:] + state[:-1]
            return new_state
        elif action == 'L':
            # 2. Cyclic shift left
            new_state = state[1:] + state[:1]
            return new_state

        new_state = state[:]
        new_state[0], new_state[1] = new_state[1], new_state[0]
        return new_state


    # Priority queue for the beam search; stores tuples of (cumulative cost, path, current state)
    queue = deque([(0, [], permutation)])
    seen = set()

    max_actions = 0
    q_values = []

    while queue:
        # Limit the size of the queue as per beam width
        queue = deque(sorted(list(queue), key=lambda x: x[0])[:beam_width])
        next_queue = deque()

        for cost, path, current in queue:
            if len(path) > max_actions:
                max_actions = len(path)
                if max_actions % 100 == 0:
                    print('new max len', len(path), len(queue), len(seen), np.min(q_values))
                q_values = []
            if len(path) > 40000:
                #print('long path', queue)
                return None
            if current == target:
                return path  # Return the path to sorted order


            for action in ['L','R','X']:
                if action == 'L' and len(path) > 0 and path[-1] == 'R':
                    continue
                if action == 'R' and len(path) > 0 and path[-1] == 'L':
                    continue
                if action == 'X' and current[0] < current[1]:
                    continue

                next_state = apply_action(current, action)

                if tuple(next_state) not in seen:
                    seen.add(tuple(next_state))

                    with torch.no_grad():
                        t_state = torch.FloatTensor(next_state).unsqueeze(0).to(device)
                        q_value = make_predictions_on_array(model, t_state.cpu().numpy() )
                        # q_value = model(t_state).squeeze().item()
                        q_values.append(q_value)

                    total_cost = cost + 1 + q_value
                    next_queue.append((total_cost, path + [action], next_state))

        queue = next_queue

    return None  # Return None if no solution is found


In [14]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class GatedResidualBlock1D(nn.Module):
    """
    A 1D TCN block with gated activation:
      - Conv1D (dilation) -> BN -> [sigmoid gate * tanh candidate] -> Dropout -> Conv1D ...
      - Residual skip
    """
    def __init__(self, in_channels, out_channels, kernel_size=3, dilation=1, dropout=0.3):
        super().__init__()
        self.out_channels = out_channels
        pad = (kernel_size - 1) * dilation // 2

        # First conv for the gated activation
        self.conv_filter = nn.Conv1d(in_channels, out_channels, kernel_size,
                                     padding=pad, dilation=dilation)
        self.conv_gate   = nn.Conv1d(in_channels, out_channels, kernel_size,
                                     padding=pad, dilation=dilation)
        self.bn_filter = nn.BatchNorm1d(out_channels)
        self.bn_gate   = nn.BatchNorm1d(out_channels)
        self.dropout = nn.Dropout(dropout)

        # Second conv
        self.conv2 = nn.Conv1d(out_channels, out_channels, kernel_size,
                               padding=pad, dilation=dilation)
        self.bn2   = nn.BatchNorm1d(out_channels)

        self.shortcut = None
        if in_channels != out_channels:
            self.shortcut = nn.Conv1d(in_channels, out_channels, 1)

    def forward(self, x):
        residual = x

        # Gated activation
        filter_out = self.conv_filter(x)
        filter_out = self.bn_filter(filter_out)
        gate_out   = self.conv_gate(x)
        gate_out   = self.bn_gate(gate_out)

        filter_out = torch.tanh(filter_out)
        gate_out   = torch.sigmoid(gate_out)
        out = filter_out * gate_out  # gating

        out = self.dropout(out)

        # Second conv
        out = self.conv2(out)
        out = self.bn2(out)

        # Residual
        if self.shortcut is not None:
            residual = self.shortcut(residual)

        out += residual
        out = F.relu(out)
        return out


class GatedTCN1D(nn.Module):
    """
    An improved TCN for 1D sequences using gated residual blocks.
      1) Embedding (if needed)
      2) Stacked GatedResidualBlock1D with dilations 1, 2, 4, ...
      3) Global pool
      4) FC for output
    """
    def __init__(self,
                 vocab_size=50,
                 embed_dim=32,
                 hidden_dim=128,
                 num_levels=4,
                 kernel_size=3,
                 dropout=0.3,
                 out_dim=1):
        super().__init__()
        self.out_dim = out_dim

        # Embedding for integer tokens
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.proj = nn.Conv1d(embed_dim, hidden_dim, kernel_size=1)

        # Build stacked blocks with growing dilation
        self.blocks = nn.ModuleList()
        dilation = 1
        for _ in range(num_levels):
            block = GatedResidualBlock1D(
                in_channels=hidden_dim,
                out_channels=hidden_dim,
                kernel_size=kernel_size,
                dilation=dilation,
                dropout=dropout
            )
            self.blocks.append(block)
            dilation *= 2

        # Global pooling
        self.global_pool = nn.AdaptiveAvgPool1d(1)

        # Final linear
        self.fc = nn.Linear(hidden_dim, out_dim)

    def forward(self, x):
        """
        x: (batch_size, seq_len) integer tokens
        """
        # 1) Embedding -> (batch, seq_len, embed_dim)
        x = self.embedding(x)
        # 2) => (batch, embed_dim, seq_len)
        x = x.permute(0, 2, 1)

        # 3) Project to hidden_dim
        x = self.proj(x)

        # 4) Pass through each gated TCN block
        for block in self.blocks:
            x = block(x)

        # 5) Global pool => (batch, hidden_dim, 1)
        x = self.global_pool(x)
        x = x.squeeze(-1)  # => (batch, hidden_dim)

        # 6) Output
        out = self.fc(x)
        if self.out_dim == 1:
            out = out.squeeze(-1)
        return out


In [15]:
import time
dimension = 16
dic={5: 20,
     10: 140,
     16: 180,
    20:	250,
21:	273,
22:	297,
23:	322,
24:	348,
25:	375,
26:	403,
27:	432,
28:	462,
29:	493,
30:	525,
31:	558,
32:	592,
35:	700,
37:	777,
40:	900,
45:	1125,
50:	1375,
60:	1950,
70:	2625,
80:	3400,
90:	4275,
100: 5250,
150:	11625,
200:	20500,
300:	45750}
n_random_walk_length = dic[dimension]
rate_of_diffusion_distance =10
n_random_walk_length = n_random_walk_length*rate_of_diffusion_distance
n_random_walk_length = 120
n_random_walks_to_generate_validation_fixed = 100
# n_random_walk_length = 40
n_random_walks_to_generate = 1000

list_generators = make_S_N(dimension)
# X_test, y_test = random_walks(n_random_walk_length,  n_random_walks_to_generate)
X_test, y_test = random_walks(list_generators, n_random_walk_length,  n_random_walks_to_generate,n_steps_back_to_ban=8)
print(X_test.shape)
test_dataset = MyIntSequenceDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

p = np.arange(dimension)
n = dimension
p[0], p[1] = p[1], p[0]
i = 2
while i < n-i+1:
    p[i], p[n-i+1] = p[n-i+1], p[i]
    i += 1
permutation = p.tolist()

torch.Size([120000, 16])


In [16]:

model = GatedTCN1D(vocab_size=dimension,
                           embed_dim=4,
                           hidden_dim=8,
                           num_levels=2,
                           kernel_size=3,
                           dropout=0.3,
                           out_dim=1).to(device)
criterion = nn.MSELoss()
optimizer = optim.AdamW(model.parameters(), lr=0.001, weight_decay=1e-5)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=2, factor=0.8, verbose=True)
pytorch_total_params = sum(p.numel() for p in model.parameters() if p.requires_grad) / 1000000
print(f'Total trainable parameters: {pytorch_total_params}')
# print(str(datetime.now()))
model, train_rmse_list, train_r2_list, test_rmse_list, test_r2_list, loss_list = train(2, f'{dimension}_GatedTCN1D')


Total trainable parameters: 0.001409
Epoch [1/2], Loss: 658.4556, Time: 2025-02-19 02:33:33.905445
Train - RMSE: 22.2457, R²: 0.5876
Test  - RMSE: 22.8087, R²: 0.5664
Epoch [2/2], Loss: 595.7567, Time: 2025-02-19 02:33:43.920266
Train - RMSE: 19.1201, R²: 0.6953
Test  - RMSE: 21.1753, R²: 0.6263
Finished Training
Best metrics: min(test_rmse_list)=21.1753 max(test_r2_list)=0.6263


In [17]:
model.to(device);
a = datetime.now()
k = 10  # Ширина "луча"
path = beam_search_sorting(permutation, beam_width=k)
if path:
    print("Кратчайший путь:", len(path))
    print(path)
else:
    print('No path found')
d = (datetime.now() - a).seconds
print(f'{d} seconds passed')


new max len 100 10 1591 43.345562
new max len 200 10 2999 28.314468
new max len 300 10 4452 10.886448
Кратчайший путь: 316
['X', 'R', 'X', 'R', 'R', 'R', 'X', 'R', 'X', 'R', 'R', 'X', 'R', 'R', 'X', 'R', 'X', 'L', 'X', 'L', 'L', 'L', 'L', 'L', 'X', 'L', 'L', 'X', 'L', 'X', 'L', 'X', 'R', 'X', 'L', 'L', 'L', 'L', 'X', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'X', 'R', 'X', 'R', 'X', 'R', 'X', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'X', 'L', 'X', 'L', 'L', 'L', 'L', 'L', 'L', 'X', 'R', 'R', 'R', 'R', 'R', 'X', 'R', 'X', 'R', 'X', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'X', 'L', 'X', 'L', 'X', 'R', 'R', 'R', 'X', 'L', 'X', 'L', 'X', 'R', 'R', 'R', 'R', 'X', 'R', 'X', 'R', 'X', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'X', 'L', 'X', 'L', 'X', 'L', 'L', 'L', 'L', 'L', 'X', 'R', 'X', 'R', 'X', 'R', 'X', 'R', 'X', 'R', 'X', 'R', 'X', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'X', 'R', 'X', 'R', 'X', 'R', 'X', 'L', 'L', 'L', 'L', 'L', 'L', 'X', 'R', 'X', 'L', 'L', 'X', 'R', 'X', 'R', 'R', 'X', '

Cloning into 'cayleypy'...
remote: Enumerating objects: 67, done.
remote: Counting objects: 100% (67/67), done.
remote: Compressing objects: 100% (44/44), done.
remote: Total 67 (delta 42), reused 47 (delta 22), pack-reused 0 (from 0)
Unpacking objects: 100% (67/67), 34.69 KiB | 740.00 KiB/s, done.


In [19]:
dimension=16
rate_of_diffusion_distance =1
CFG = { 'dimension'                       :    dimension,     # N
        'generators_family'               : 'S_N',
        'n_epochs'                        :  1000//rate_of_diffusion_distance *5,     #41,
        'epochs_start'                    :  0,
        'lr'                              :  0.000001,
        'range_of_cubies'                 :  100 ,
       
        ### adavanced beam search params
        'beam_width'                      : 2**17 , #100k
        'random_walks'                    : 10000 ,   #10k
       
        'n_steps_limit'                   : dimension*rate_of_diffusion_distance*4   ,
        'n_step_size'                     : 1     , 
        'n_beam_candidate_states'         : 'Auto',
        'radius_destination_neigbourhood' : 5     ,
        'radius_beam_neigbourhood'        : 0     , # not used actually
        'bi_bfs_chunk_size_states'        : 2**16 , # not used actually  2**16
        'mode_bibfs_checks'               : (10,5), # not used actually
        'temperature'                     : 0     ,
        'temperature_decay'               : 1     ,
        'alpha_past_states_attenuation'   : 0     ,
        'n_steps_to_ban_backtracking'     : 100     ,
        'flag_empty_backtracking_list'    : False ,
        'n_attempts_limit'                : 1     ,
        'do_check_stagnation'             : False , 
        'stagnation_steps'                : 0    ,
        'stagnation_thres'                : 0.005 ,
        'n_random_start_steps'            : 0     ,
        'diversity_func'                  : None,
        'diversity_weight'                : None     ,
        'sub_ray_split'                   : False ,
        #'random_seed'                     : -23057572


        ### train params
        #'n_random_walk_length'                                :    ,
        'n_random_walks_to_generate'                          : 1000,
        'n_random_walks_to_generate_validation_new_each_epoch': 100 ,
        'n_random_walks_to_generate_validation_fixed'         : 100 ,
        'batch_size'                                          : 256 ,
        'list_epochs_to_save_predictions_and_errors'          : [1,2,4,8,16,32,64,100],
      }

dic={16:	168,
17:	187,
18:	207,
19:	228,
20:	250,
21:	273,
22:	297,
23:	322,
24:	348,
25:	375,
26:	403,
27:	432,
28:	462,
29:	493,
30:	525,
31:	558,
32:	592,
33:	627,
34:	663,
35:	700,
36:	738,
37:	777,
38:	817,
39:	858,
40:	900,
41:	943,
42:	987,
43:	1032,
44:	1078,
45:	1125,
46:	1173,
47:	1222,
48:	1272,
49:	1323,
50:	1375}
n_random_walk_length=dic[dimension]
CFG['n_random_walk_length'] = n_random_walk_length #(CFG['dimension']*(CFG['dimension']-1)//2 ) + CFG['dimension']*3
print(f"Version:{dimension},e{CFG['epochs_start']}+{1000//rate_of_diffusion_distance}={CFG['epochs_start']+1000//rate_of_diffusion_distance},r{n_random_walk_length*rate_of_diffusion_distance}.")

Version:16,e0+1000=1000,r168.


In [20]:
1

1

In [21]:
def state_from_code(N, moves):
    
        moves_cleanup = [(q[0],q[1]) if q[0]<q[1] else (q[1],q[0]) for q in moves if q[0]!=q[1]]
        moves_cleanup = list(dict.fromkeys(moves_cleanup,None).keys())
        
        res = list(range(N))
        for i,j in moves_cleanup:
            res[i],res[j] = res[j],res[i]
    
        return res
    
sts1 = state_from_code(CFG['dimension'], moves = [(0,1),]+[(q+2,CFG['dimension']-q-1) for q in range(CFG['dimension']//2)])
    

In [22]:
sts = sts1
params  = [(0,0)] 

In [23]:
# from cayleypy.cayleypy import *
# def make_S_N(N):
#     return [
#         [1,0,]+[q+2 for q in range(N-2)]       ,
#                [q+1 for q in range(N-1)] + [0,],
#         [N-1,]+[q+0 for q in range(N-1)]       ,
#     ]

# def make_permutohedron(N):
#     return [
#         list(range(0,i)) + [i+1,i,] + list(range(i+2,N)) for i in range(N-1)
#     ]# + [[N-1,]+list(range(1,N-1))+[0,]]

# def make_all_pair_perms(N):
#     res = []

#     for i in range(N):
#         for j in range(N):
#             candidate = list(range(N))
#             if i!=j:
#                 candidate[i] = j
#                 candidate[j] = i
#                 res.append(tuple(candidate))
    
#     return res

# def make_pancake(N):
#     return [
#         list(range(i,-1,-1)) + list(range(i+1, N)) for i in range(1,N)
#     ]
# def is_1_neighbour(cg, state1, state2):
#     state1_neighbours = get_neighbors2(state1, cg.tensor_generators )
#     for i in range(state1_neighbours.shape[0]):
#         if torch.allclose(state1_neighbours[i,:], state2):
#             return i, state1_neighbours[i:i+1,:]
#     return -1, None

# def make_unique_path_from_path_states(cg, path_states):
#     res_mvs = []
#     res_sts = [path_states[0][0:1,:].detach().clone(),]
    
#     ps0 = path_states[0][0:1,:].detach().clone()
#     for ps1 in path_states[1:]:
#         for ps1_i in range(ps1.shape[0]):
#             m,s = is_1_neighbour(cg, ps0, ps1[ps1_i:ps1_i+1],)
#             if m>-1:
#                 res_mvs.append(m)
#                 res_sts.append(s.detach().clone())
#                 ps0 = s.detach().clone()
#                 break

#     return torch.vstack(res_sts), res_mvs

# families = {'S_N':make_S_N,
#             'PHD':make_permutohedron,
#             'PRS':make_all_pair_perms,
#             'PNC':make_pancake,}
# list_generators = families[CFG['generators_family']](CFG['dimension'])
# cayley_group = CayleyGraph( list_generators , dtype=torch.long)
# cayley_group.state_destination

In [24]:
path_return=False
model = model.half()
model.dtype = torch.float16

In [25]:
# dict_additional_data = dict()

# rdn, nbt = params[0] 

# flag_found_destination, i_step, dict_additional_data  =\
#      cayley_group.beam_search_permutations_torch(
#          state_start                     = sts                                   ,
#          models_or_heuristics            = model                                ,

#          beam_width                      = CFG ['beam_width'],
#          n_steps_limit                   = CFG ['dimension']*CFG ['dimension'] + CFG ['dimension']*3, #1700,#min(A[enum]//2, 600), #CFG["n_steps_limit"                  ],
#          n_step_size                     = CFG["n_step_size"                    ],
#          n_beam_candidate_states         = CFG["n_beam_candidate_states"        ],

#          radius_destination_neigbourhood = 21, #rdn, #CFG["radius_destination_neigbourhood"],

#          radius_beam_neigbourhood        = CFG["radius_beam_neigbourhood"       ],
#          bi_bfs_chunk_size_states        = CFG["bi_bfs_chunk_size_states"       ],
#          mode_bibfs_checks               = CFG["mode_bibfs_checks"              ],
#          temperature                     = CFG["temperature"                    ],
#          alpha_past_states_attenuation   = CFG["alpha_past_states_attenuation"  ],

#          n_steps_to_ban_backtracking     = nbt, #CFG["n_steps_to_ban_backtracking"    ],

#          flag_empty_backtracking_list    = CFG["flag_empty_backtracking_list"   ],
#          n_attempts_limit                = CFG["n_attempts_limit"               ],
#          do_check_stagnation             = CFG["do_check_stagnation"            ],
#          stagnation_steps                = CFG["stagnation_steps"               ],
#          stagnation_thres                = CFG["stagnation_thres"               ],
#          n_random_start_steps            = CFG["n_random_start_steps"           ],
#          diversity_func                  = CFG['diversity_func'                 ],
#          diversity_weight                = CFG['diversity_weight'               ],
#          sub_ray_split                   = CFG['sub_ray_split'                  ],

#          verbose                         = -1,
#          random_seed                     = dict_additional_data.get('random_seed',None),
#          flag_path_return                = path_return, #False,#True,
#        )




# if flag_found_destination:
#     if path_return==True:
#         A, path_moves = make_unique_path_from_path_states(cayley_group, dict_additional_data["path_states"] )
#         moves_mapping = {0:'X',1:'R',2:'L'}
#         moves_res = ''.join(reversed([moves_mapping[q] for q in path_moves]))
#     else: 
#         moves_res = ''
# #     print(       f'True: epoch:{epoch};   flag_found_destination: {flag_found_destination}; i_step: {i_step};  beam_width:{CFG ["beam_width"]}; dim:{CFG["dimension"]};;moves_res:{moves_res}')
# #     logging.info(f'True: epoch:{epoch}:   flag_found_destination: {flag_found_destination}; i_step: {i_step};  beam_width:{CFG ["beam_width"]}; dim:{CFG["dimension"]}; seed:{dict_additional_data.get('random_seed',None)};moves_res:{moves_res}')
# # else:
# #     print(       f'False: epoch:{epoch};  flag_found_destination: {flag_found_destination}; i_step: {7777777};  beam_width:{CFG ["beam_width"]}; dim:{CFG["dimension"]}; seed:{dict_additional_data.get('random_seed',None)}; sts: {sts};')                                        
# #     logging.info(f'False: epoch:{epoch};  flag_found_destination: {flag_found_destination}; i_step: {7777777};  beam_width:{CFG ["beam_width"]}; dim:{CFG["dimension"]}; seed:{dict_additional_data.get('random_seed',None)}; sts: {sts};')   
